In [3]:
import pandas as pd
from pathlib import Path

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import tiktoken

In [4]:
image_comments_folder = Path("/Users/michaelbai/ML/dataset/flickr30k_images")
image_comments_file = image_comments_folder / "results.csv" 
images_folder = image_comments_folder / "flickr30k_images"
print(image_comments_file)
list(images_folder.glob("*.jpg"))[:2]

/Users/michaelbai/ML/dataset/flickr30k_images/results.csv


[PosixPath('/Users/michaelbai/ML/dataset/flickr30k_images/flickr30k_images/2609797461.jpg'),
 PosixPath('/Users/michaelbai/ML/dataset/flickr30k_images/flickr30k_images/1788892671.jpg')]

In [5]:
df = pd.read_csv(image_comments_file, sep="|", index_col=False)
df = df.astype({'image_name': 'str', 'comment_number': int, 'comment': str})
df1 = df[0: 1]
list(df1["image_name"]), df1["image_name"][0]

(['1000092795.jpg'], '1000092795.jpg')

In [6]:
test_img = Image.open("/Users/michaelbai/ML/dataset/flickr30k_images/flickr30k_images/1000092795.jpg")
convert_tensor = transforms.ToTensor()
convert_tensor(test_img)

tensor([[[0.0000, 0.0078, 0.0118,  ..., 0.8588, 0.8706, 0.8667],
         [0.0157, 0.0118, 0.0196,  ..., 0.8941, 0.8314, 0.8549],
         [0.0118, 0.0157, 0.0196,  ..., 0.9490, 0.9098, 0.8392],
         ...,
         [0.6510, 0.7059, 0.5098,  ..., 0.4157, 0.5255, 0.4196],
         [0.7922, 0.5647, 0.5216,  ..., 0.6157, 0.6118, 0.6157],
         [0.5647, 0.7255, 0.7216,  ..., 0.6118, 0.7490, 0.4157]],

        [[0.0235, 0.0235, 0.0196,  ..., 0.9569, 0.9922, 0.9765],
         [0.0314, 0.0196, 0.0275,  ..., 0.9922, 0.9608, 0.9922],
         [0.0118, 0.0157, 0.0196,  ..., 1.0000, 1.0000, 0.9608],
         ...,
         [0.6941, 0.7373, 0.6078,  ..., 0.6196, 0.6745, 0.6392],
         [0.8588, 0.6863, 0.6275,  ..., 0.7373, 0.7294, 0.7686],
         [0.6157, 0.8275, 0.8392,  ..., 0.7294, 0.7843, 0.4941]],

        [[0.0157, 0.0196, 0.0157,  ..., 0.9765, 1.0000, 0.9804],
         [0.0275, 0.0157, 0.0235,  ..., 0.9804, 0.9961, 1.0000],
         [0.0118, 0.0157, 0.0196,  ..., 1.0000, 0.9843, 0.

In [7]:
class Config:
    # Image
    img_width_size = 400
    img_height_size = 400

    # Text
    max_text_len = 50

def load_image_tensor(config: Config, img_file_path: Path) -> torch.tensor:
    # Load image from file
    img = Image.open(img_file_path)
    print()
    print(f"img: {img}")
    
    # Convert to tensor
    convert_tensor = transforms.ToTensor()
    image_tensor = convert_tensor(img)

    # resize base on config
    resize = transforms.Resize(size=(config.img_height_size, config.img_width_size))
    image_tensor = resize(image_tensor)
    return image_tensor

# Create Dataset 
class ImgCommentDataset(Dataset):
    def __init__(self, config: Config,  img_comments_folder: Path, train_test_split: str = "train", train_test_split_portion: float = 0.8):
        self.config = config
        self.img_commments_folder = img_comments_folder
        self.img_comments_file = img_comments_folder / "results.csv"
        self.imgs_folder = img_comments_folder / "flickr30k_images"
        
        self.train_test_split = train_test_split
        self.train_test_split_portion = train_test_split_portion

        # The current `results.csv` file is using "| " to seperate 3 columns. 
        # For the pd.read_csv, the `sep` here is given as a regular expression. 
        df = pd.read_csv(self.img_comments_file, sep="|")
        train_split_len = int(len(df)*self.train_test_split_portion)
        if self.train_test_split == "train":
            self.img_comments_df = df[:train_split_len]
        else:
            self.img_comments_df = df[train_split_len:]
        
        self.text_encoder = tiktoken.encoding_for_model("gpt-4o")
            
    def __len__(self):
        return len(self.img_comments_df)
    
    def __getitem__(self, idx:int):
        print(f"idx: {idx}")
        row_df = self.img_comments_df[idx: idx+1]
        image_name = str(list(row_df["image_name"])[0])
        assert (self.imgs_folder/image_name).is_file(), f"cannot find file: {self.img_commments_folder/image_name}"

        comment_number = int(list(row_df["comment_number"])[0])
        comment = str(list(row_df["comment"])[0])
        comment_encoding = self.text_encoder.encode(comment)
        if len(comment_encoding) > self.config.max_text_len:
            comment_encoding = comment_encoding[:self.config.max_text_len]
        else:
            # TODO: review append `<|im_end|>` - 200265 logic
            comment_encoding = comment_encoding + [200265 for _ in range(self.config.max_text_len - len(comment_encoding))]
        assert len(comment_encoding) == self.config.max_text_len
        comment_encoding = torch.tensor(comment_encoding, dtype=torch.int)
        
        # return load_image_tensor(self.imgs_folder/image_name), comment_number, comment, comment_encoding
        image_tensor = load_image_tensor(self.config, self.imgs_folder/image_name)

        return image_tensor, comment_encoding

In [8]:
config = Config()

train_dataset = ImgCommentDataset(config, image_comments_folder, train_test_split="train", train_test_split_portion=0.8)
test_dataset = ImgCommentDataset(config, image_comments_folder, train_test_split="test", train_test_split_portion=0.8)
print(len(train_dataset), len(test_dataset))

127132 31783
